# Derive elevation map features from LiDAR scans

Models used in the original experiments:
- [ ] DEM (Digital Elevation Model) / LiDAR
- [ ] DEM / height
- [x] slope
- [ ] aspect
- [x] hillside
- [ ] roughness
- [ ] profile curvature
- [ ] tangential curvature
- [ ] valley depth
- [ ] TWI (Topographic Wetness Index)
- [ ] TPI (Topographic Position Index)
- [ ] TRI (Topographic Ruggedness Index)
- [ ] catchment area

Process works in two stages:
1. Compute derived features in TIFF native type like `double` or `int64`
2. Convert native TIFF into viewable grayscale image, using log scaling OR robust histogram-based normalization


### Parallel limited concurrency shell

```
N=4
(
for thing in a b c d e f g; do 
   ((i=i%N)); ((i++==0)) && wait
   sleep 1 & echo "$thing" & 
done
)
```

## Elevation model - multiple files

In [2]:
lidar_10m = "/Users/akusok/wrkdir/GEODATA/National Land Survey of Finland, Elevation model, 10 m x 10 m, 2019, TIFF"
# lidar_2m = "/Users/akusok/wrkdir/GEODATA/National Land Survey of Finland, Elevation model, 2 m x 2 m, 2008-2020, TIFF"

lidar_2m = "/Users/akusok/wrkdir/elev-model-2m"

**TODO: fixed scaling**

### hillshades

Code should work from the TIFF folder

In [ ]:
%%sh

dirname=hillshade
mkdir ../ELEVATION/$dirname
N=8
time (
for name in $(find . -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $name & gdaldem hillshade ${name} ../ELEVATION/${dirname}/${name:2} -q & 
    done
)

In [ ]:
%%sh

dirname=hillshade_combined
mkdir ../ELEVATION/$dirname
N=8
time (
for name in $(find . -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $name & gdaldem hillshade -combined ${name} ../ELEVATION/${dirname}/${name:2} -q & 
    done
)

In [ ]:
%%sh

dirname=hillshade_multidirectional
mkdir ../ELEVATION/$dirname
N=8
time (
for name in $(find . -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $name & gdaldem hillshade -multidirectional ${name} ../ELEVATION/${dirname}/${name:2} -q & 
    done
)

Convert to tiles

Use `conda deactivate` environment for a working GDAL

Resampling methods: default (no flag) is fast and good, `lanczos` is better but much slower

In [ ]:
%%sh

dirname=hillshade
gdalbuildvrt ${dirname}.vrt ./${dirname}/*.tif
mkdir TILES_${dirname}
time gdal2tiles.py -e -z 5-16 --processes=8 ${dirname}.vrt ./TILES_${dirname}

In [ ]:
%%sh

dirname=hillshade_combined
gdalbuildvrt ${dirname}.vrt ./${dirname}/*.tif
mkdir TILES_${dirname}
time gdal2tiles.py -e -z 5-16 --processes=8 ${dirname}.vrt ./TILES_${dirname}

In [ ]:
%%sh

dirname=hillshade_multidirectional
gdalbuildvrt ${dirname}.vrt ./${dirname}/*.tif
mkdir TILES_${dirname}
time gdal2tiles.py -e -z 5-16 --processes=8 ${dirname}.vrt ./TILES_${dirname}

### Slope

In [ ]:
%%sh

dirname=slope
mkdir ../ELEVATION/$dirname
mkdir ../ELEVATION/tmp
N=8
(
for name in $(find . -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $name & 
    gdaldem slope -p ${name} ../ELEVATION/tmp/${name:2} -q &
    done
)
(
for name in $(find . -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    gdal_translate -ot Byte -scale -exponent 1 -q ../ELEVATION/tmp/${name:2} ../ELEVATION/${dirname}/${name:2} &
    done
)
rm -rf ../ELEVATION/tmp

In [ ]:
%%sh

dirname=slope
gdalbuildvrt ${dirname}.vrt ./${dirname}/*.tif
mkdir TILES_${dirname}
time gdal2tiles.py -e -r lanczos --processes=8 ${dirname}.vrt ./TILES_${dirname}

### Universal functions

Does two processing steps with optional `tmp` folder between them.

Trick: `scale` applies different scaling to every image and breaks transformation. Avoid by setting `-scale min max` params, **OR** by merging all images together into a Virtual Raster Tile and apply scaling to this *one* virtual image.

**Hillshade**

In [ ]:
%%sh

feature=hillshade
datadir=../elev-model-2m/
mkdir $dirname
N=4
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename $name) & 
    gdaldem hillshade ${name} ./${feature}/$(basename ${name}) -q & 
    done
)
sleep 3
gdalbuildvrt ${feature}.vrt ./${feature}/*.tif
mkdir TILES_${feature}
gdal2tiles.py --processes=$N ${feature}.vrt ./TILES_${feature}

**Slope**

In [ ]:
%%sh

feature=slope
datadir=../elev-model-2m/
mkdir $feature
mkdir TILES_${feature}
N=4
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename ${name}) & 
    gdaldem slope -p ${name} ./${feature}/$(basename ${name}) -q &
    done
)
sleep 1
gdalbuildvrt ${feature}_raw.vrt ./${feature}/*.tif
gdal_translate -ot Byte -scale -exponent 0.7 -a_nodata 255 -q ${feature}_raw.vrt ${feature}.vrt
time gdal2tiles.py --processes=$N ${feature}.vrt ./TILES_${feature}

**Aspect**

In [ ]:
%%sh

feature=aspect
datadir=../elev-model-2m/
mkdir $feature
mkdir TILES_${feature}
N=4
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename ${name}) & 
    gdaldem aspect ${name} ./${feature}/$(basename ${name}) -q &
    done
)
sleep 1
gdalbuildvrt ${feature}_raw.vrt ./${feature}/*.tif
gdal_translate -ot Byte -scale 0 360 -q ${feature}_raw.vrt ${feature}.vrt
time gdal2tiles.py --processes=$N ${feature}.vrt ./TILES_${feature}

**Terrain Ruggedness Index (TRI)**

In [ ]:
%%sh

feature=tri
datadir=../elev-model-2m/
mkdir $feature
mkdir TILES_${feature}
N=4
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename ${name}) & 
    gdaldem TRI ${name} ./${feature}/$(basename ${name}) -q &
    done
)
sleep 1
gdalbuildvrt ${feature}_raw.vrt ./${feature}/*.tif
gdal_translate -ot Byte -scale 0 30 -exponent 0.5 -a_nodata 255 -q ${feature}_raw.vrt ${feature}.vrt
time gdal2tiles.py --processes=$N ${feature}.vrt ./TILES_${feature}

**Terrain Position Index (TPI)**

We can hide no-data value with `--hideNoData` to avoid having holes all around the map

In [ ]:
%%sh

feature=tpi
datadir=../elev-model-2m/
mkdir $feature
mkdir TILES_${feature}
N=4
mkdir tmp
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename ${name}) & 
    gdaldem TPI ${name} ./tmp/$(basename ${name}) -q &
    done
)
sleep 1
(
for name in $(find ./tmp -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename ${name}) & 
    gdal_calc.py -A tmp/$(basename ${name}) --outfile=./${feature}/$(basename ${name}) --calc="-40*numpy.log10(A*(A>=1e-3) + 1*(A<1e-3)) + 40*numpy.log10(-A*(A<=-1e-3) + 1*(A>-1e-3))" --NoDataValue=0 --quiet &
    done
)
sleep 1
rm -rf ./tmp
gdalbuildvrt ${feature}_raw.vrt ./${feature}/*.tif
gdal_translate -ot Byte -scale -127 128 -a_nodata 255 -q ${feature}_raw.vrt ${feature}.vrt
time gdal2tiles.py -z 5-17 --processes=$N ${feature}.vrt ./TILES_${feature}

In [ ]:
    # gdal_calc.py -A ./tmp/$(basename ${name}) --outfile=./${feature}/$(basename ${name}) --calc="-40*numpy.log10(A*(A>=1e-3) - A*(A<=-1e-3) + 1*logical_and(A>-1e-3,A<1e-3))" --NoDataValue=0 -q &
    
    

Manual TPI with configurable size kernel

In [12]:
from scipy import ndimage
from PIL import Image
import rasterio
import numpy as np
from scipy.signal import fftconvolve, oaconvolve
from shutil import copy

def circle(diameter):
    mid = (diameter - 1) / 2
    distances = np.indices((diameter, diameter)) - np.array([mid, mid])[:, None, None]
    kernel = ((np.linalg.norm(distances, axis=0) - mid) <= 0.5).astype(float)

    return kernel

In [91]:
tif = rasterio.open("source.tif")

In [225]:
k = 15

kernel = circle(k)
kernel[k//2, k//2] = 0
kernel /= -np.sum(kernel)
kernel[k//2, k//2] = 1

with rasterio.open("s2.tif", "w", **tif.profile) as f:
    f.write( ndimage.convolve(tif.read(1), kernel, mode="mirror")[None, :, :] )

In [247]:
k = 5

while k < 500:
    print(k, end=', ')
    kernel = circle(k)
    kernel[k//2, k//2] = 0
    kernel /= -np.sum(kernel)
    kernel[k//2, k//2] = 1

    copy("source.tif", f"convolve_{k:03d}.tif")
    
    with rasterio.open(f"convolve_{k:03d}.tif", "w", **tif.profile) as f:
        f.write( oaconvolve(tif.read(1), kernel, mode="same")[None, :, :] )
    
    k = int(k * 1.3)
    if k%2 == 0:
        k += 1

5, 7, 9, 11, 15, 19, 25, 33, 43, 55, 71, 93, 121, 157, 205, 267, 347, 451, 

**Big One**

In [15]:
vrt = rasterio.open("/Users/akusok/wrkdir/elev-model-2m/elevation-2m.vrt")

In [28]:
vrt.profile

{'driver': 'VRT', 'dtype': 'float32', 'nodata': -9999.0, 'width': 30000, 'height': 24000, 'count': 1, 'crs': CRS.from_epsg(3067), 'transform': Affine(2.0, 0.0, 350000.0,
       0.0, -2.0, 6702000.0), 'blockxsize': 128, 'blockysize': 128, 'tiled': True}

In [30]:
from rasterio.windows import Window

In [32]:
for i,w in enumerate(vrt.block_windows()):
    print(w)
    if i > 5:
        break

((0, 0), Window(col_off=0, row_off=0, width=128, height=128))
((0, 1), Window(col_off=128, row_off=0, width=128, height=128))
((0, 2), Window(col_off=256, row_off=0, width=128, height=128))
((0, 3), Window(col_off=384, row_off=0, width=128, height=128))
((0, 4), Window(col_off=512, row_off=0, width=128, height=128))
((0, 5), Window(col_off=640, row_off=0, width=128, height=128))
((0, 6), Window(col_off=768, row_off=0, width=128, height=128))


In [42]:
vrt.read(window=Window(0, 0, 3100, 3100)).shape

(1, 3100, 3100)

## >>>

**Roughness**

In [ ]:
%%sh

feature=roughness
datadir=../elev-model-2m/
mkdir ./tmp
N=4
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename ${name}) & 
    gdaldem roughness ${name} ./tmp/$(basename ${name}) -q &
    done
)
sleep 3
mkdir $feature
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    gdal_translate -ot Byte -scale -a_nodata 255 -q ./tmp/$(basename ${name}) ./${feature}/$(basename ${name}) &
    done
)
sleep 3
rm -rf ./tmp
gdalbuildvrt ${feature}.vrt ./${feature}/*.tif
mkdir TILES_${feature}
time gdal2tiles.py --processes=$N ${feature}.vrt ./TILES_${feature}

In [ ]:
%%sh

wget --show-progress -x --cut-dirs=3 -i paituli_78876364.txt

In [ ]:
%%sh

for name in $(find . -name "*.TIF"); do echo $name;  gdal_translate -of VRT -ot Byte -scale 0 14000 -exponent 0.25 ./${name} ${name}.vrt; done

In [ ]:
%%sh

gdalbuildvrt ../all.vrt *.TIF.vrt
gdal_translate -of VRT -ot Byte ../elevation-2m.vrt ../temp.vrt

test scaling

`gdalinfo -hist temp.vrt`

In [ ]:
%%sh

gdal2tiles.py --zoom=1-9 --xyz -e --processes=8 all.vrt /Users/akusok/wrkdir/TILES/elevation_25m

In [1]:
from PIL import Image

In [21]:
# img = Image.open("/Users/akusok/wrkdir/MapTiles/Geologia/R5241E.tif")
img = Image.open("/Users/akusok/wrkdir/MapTiles/Geologia/PAITULI_elevation_25m/2000/30Q_DEM2.TIF")

In [14]:
import numpy as np

In [23]:
x = np.array(img).ravel()
# np.histogram(x, bins=np.arange(280))
list(np.unique(x))

[0,
 327,
 1124,
 1125,
 1128,
 1129,
 1132,
 1138,
 1140,
 1147,
 1151,
 1152,
 1154,
 1166,
 1167,
 1168,
 1169,
 1170,
 1177,
 1178,
 1179,
 1185,
 1189,
 1194,
 1195,
 1197,
 1198,
 1199,
 1200,
 1201,
 1202,
 1203,
 1204,
 1205,
 1206,
 1207,
 1208,
 1209,
 1210,
 1211,
 1212,
 1213,
 1214,
 1215,
 1216,
 1217,
 1218,
 1219,
 1220,
 1221,
 1222,
 1223,
 1224,
 1225,
 1226,
 1227,
 1228,
 1229,
 1230,
 1231,
 1232,
 1233,
 1234,
 1235,
 1236,
 1237,
 1238,
 1239,
 1240,
 1241,
 1242,
 1243,
 1244,
 1245,
 1246,
 1247,
 1248,
 1249,
 1250,
 1251,
 1252,
 1253,
 1254,
 1255,
 1256,
 1257,
 1258,
 1259,
 1260,
 1261,
 1262,
 1263,
 1264,
 1265,
 1266,
 1267,
 1268,
 1269,
 1270,
 1271,
 1272,
 1273,
 1274,
 1275,
 1276,
 1277,
 1278,
 1279,
 1280,
 1281,
 1282,
 1283,
 1284,
 1285,
 1286,
 1287,
 1288,
 1289,
 1290,
 1291,
 1292,
 1293,
 1294,
 1295,
 1296,
 1297,
 1298,
 1299,
 1300,
 1301,
 1302,
 1303,
 1304,
 1305,
 1306,
 1307,
 1308,
 1309,
 1310,
 1311,
 1312,
 1313,
 1314,
 13

In [ ]:
%%sh

feature=slope_cut
datadir=../elev-model-2m/
mkdir $feature
mkdir TILES_cut_${feature}
N=4
(
for name in $(find ${datadir} -name "*.tif"); do
    ((i=i%N)); ((i++==0)) && wait
    echo $(basename ${name}) & 
    gdaldem slope -p ${name} ./${feature}/$(basename ${name}) -q &
    done
)
sleep 1
gdalbuildvrt ${feature}_raw.vrt ./${feature}/*.tif
gdal_translate -ot Byte -scale -exponent 0.7 -a_nodata 255 -q ${feature}_raw.vrt ${feature}.vrt

gdalwarp -cutline ../littorina_coast_mask_1km_buffer/littorina_coast_mask_1km_buffer.shp -crop_to_cutline -dstalpha ${feature}.vrt ${feature}-cut.vrt

time gdal2tiles.py --zoom=2-9 --xyz --processes=$N ${feature}-cut.vrt ./TILES_${feature}

In [ ]:
gdalbuildvrt littorina.vrt ../littorina_coast_mask_1km_buffer/littorina_coast_mask_1km_buffer_20m_raster.tif

gdal_translate -of vrt -expand rgba littorina.vrt littorina-rgba.vrt

time gdal2tiles.py --zoom=2-15 --xyz --processes=8 littorina-rgba.vrt ./TILES_littorina